In [ ]:
!pip install tiktoken 


   ---------------------------------------- 0.0/894.2 kB ? eta -:--:--
   --------------------------------------- 894.2/894.2 kB 10.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import groq
import time
import tiktoken  

# Function to count tokens using TikToken
def count_tokens(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")  # Use the right tokenizer for Groq
    return len(tokenizer.encode(text))

# Scrape Wikipedia page
def scrape_wikipedia_page(url):
    """Scrapes the full text from a Wikipedia page."""
    response = requests.get(url)
    if response.status_code != 200:
        return f"Could not retrieve content from {url}"
    
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.find_all("p")
    full_text = "\n".join([p.get_text() for p in paragraphs])

    return full_text.strip()

# Split text into smaller chunks (Max: 500 tokens each)
def chunk_text(text, max_tokens=500):
    words = text.split()
    chunks = []
    current_chunk = []
    current_tokens = 0

    for word in words:
        token_count = count_tokens(word)
        if current_tokens + token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_tokens = token_count
        else:
            current_chunk.append(word)
            current_tokens += token_count

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks




In [3]:
# Initialize Groq client
client = groq.Client(api_key="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")  

# System instruction for LLM
system_message = "You are an expert on historical monuments. Provide clear and informative responses."

def retrieve_from_text(query, text):
    """Retrieves relevant information from the extracted Wikipedia text."""
    
    # Split text into smaller chunks
    text_chunks = chunk_text(text, max_tokens=500)
    
    # Process each chunk separately
    relevant_responses = []
    
    for chunk in text_chunks:
        prompt = f"Context: {chunk}\n\nUser Query: {query}"
        
        try:
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ]
            )
            relevant_responses.append(response.choices[0].message.content)
            time.sleep(1)  # Respect API rate limits
        except groq.exceptions.APIStatusError as e:
            print(f"Error processing chunk: {e}")
            break

    return " ".join(relevant_responses)

# Scrape Wikipedia (happens once)
wiki_url = "https://en.wikipedia.org/wiki/Hampi"
context_text = scrape_wikipedia_page(wiki_url)

# User query
user_query = input("Enter your query: ")  

# Retrieve relevant information from the stored text
retrieved_info = retrieve_from_text(user_query, context_text)

# Print the model's response
print("\nGenerated Response:\n")
print(retrieved_info)


Generated Response:

A very significant and tragic aspect of Hampi's history! The destruction of Hampi, the capital of the Vijayanagara Empire, is a well-documented and deeply regrettable event in Indian history.

In 1565, Hampi was attacked and pillaged by a coalition of Muslim Sultanates, specifically the Deccan Sultanates of Bijapur, Golconda, Ahmadnagar, and Bidar. This devastating event marked the end of the Vijayanagara Empire, which was one of the most powerful and prosperous empires in Indian history.

Here are some key facts about the destruction of Hampi:

1. **The Battle of Talikota**: The Muslim Sultanates, led by Hussain Nizam Shah I of Ahmadnagar, Ali Adil Shah I of Bijapur, Ibrahim Quli Qutb Shah of Golconda, and Ali Barid Shah of Bidar, formed an alliance to defeat the Vijayanagara Empire. The two armies clashed at the Battle of Talikota, which took place on January 23, 1565, about 100 km (62 miles) north of Hampi.
2. **Defeat of the Vijayanagara Empire**: The Vijayana